In [1]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import sys
from keras.models import load_model
import os
import random
import math
import numpy as np
from imutils import paths

from keras.models import Sequential,Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D,Conv3D,Conv1D
from keras.layers.convolutional import MaxPooling2D,MaxPooling3D,MaxPooling1D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import LSTM,Input,GlobalAveragePooling1D
from keras import backend as K
from keras.optimizers import Adam,SGD
from keras.layers.wrappers import TimeDistributed, Bidirectional
import tensorflow as tf
from keras.callbacks import EarlyStopping,ModelCheckpoint

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
#import helpers
from Audio import Audio

In [3]:
audio = Audio(path='audio/save/')
trainlist=audio.generate_audio_list(p='train')

In [4]:
#valX,valY = audio.load_audio(p='val')
vallist = audio.generate_audio_list(p='val')

# audio model

In [8]:
class_num = 500
input_audio = Input(shape = (26,121))

conv1d_1 = Conv1D(64,kernel_size = 3,strides=1, padding = 'same',
               activation = 'relu')(input_audio)

batch_norm1 = BatchNormalization()(conv1d_1)

batch_norm1 = MaxPooling1D(pool_size=2,padding = 'same')(batch_norm1)

conv1d_2 = Conv1D(128,kernel_size = 3, strides = 1, padding = 'same',
               activation = 'relu')(batch_norm1)

batch_norm2 = BatchNormalization()(conv1d_2)
batch_norm2 = MaxPooling1D(pool_size=2,padding = 'same')(batch_norm2)

conv1d_2 = Conv1D(512,kernel_size = 3, strides = 1, padding = 'same',
               activation = 'relu')(batch_norm2)

batch_norm2 = BatchNormalization()(conv1d_2)
batch_norm2 = MaxPooling1D(pool_size=2,padding = 'same')(batch_norm2)

bilstm1 = Bidirectional(LSTM(256,return_sequences=True,name='bilstm1'))(batch_norm2)
#bilstm2 = Bidirectional(LSTM(256,return_sequences=True),name = 'bilstm2')(bilstm1)

avgpool = GlobalAveragePooling1D(name='audio_avg')(bilstm1)

dense1 = Dense(512,activation='relu')(avgpool)
batch_norm3 = BatchNormalization()(dense1)
dense2 = Dense(class_num,activation = 'softmax')(batch_norm3)
#maxpooling1d = MaxPooling1D(pool_size=2, padding='same')(batch_norm2)



Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [9]:
model = Model(inputs=[input_audio], 
              outputs=[dense2])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 26, 121)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 26, 64)            23296     
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 64)            256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 128)           24704     
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 128)           512       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7, 128)            0         
__________

In [10]:
print("[INFO] compiling model...")
sgd = SGD(lr=0.003, decay=1e-6, momentum=0.9)
#adam = Adam(lr = 0.003,decay=1e-6)
model.compile(loss="categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"])

[INFO] compiling model...


# training

In [14]:
print("[INFO] training network...")
#print(sample_number)
filepath="audio_weights/second-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
mode='max')
stopper = EarlyStopping(monitor='val_acc',patience=3)
callbacks_list = [checkpoint,stopper]

[INFO] training network...


In [15]:
sample_number = len(trainlist)
val_number = len(vallist)
batch = 64
history=model.fit_generator(audio.generate_arrays_from_file(filelist=trainlist),
                        validation_data=
                    audio.generate_arrays_from_file(filelist=vallist),
                        steps_per_epoch=sample_number // batch,
                        validation_steps = val_number//batch,
                        epochs=10, verbose=1,
                   callbacks = callbacks_list)

Epoch 1/10
7636/7636 [==============================] - 14560s 2s/step - loss: 1.8987 - acc: 0.5719 - val_loss: 1.0725 - val_acc: 0.7196

Epoch 00001: val_acc improved from -inf to 0.71955, saving model to audio_weights/second-improvement-01-0.72.hdf5
Epoch 2/10
7636/7636 [==============================] - 13416s 2s/step - loss: 0.7867 - acc: 0.7896 - val_loss: 0.8226 - val_acc: 0.7783

Epoch 00002: val_acc improved from 0.71955 to 0.77833, saving model to audio_weights/second-improvement-02-0.78.hdf5
Epoch 3/10
7636/7636 [==============================] - 13546s 2s/step - loss: 0.5523 - acc: 0.8495 - val_loss: 0.7443 - val_acc: 0.7986

Epoch 00003: val_acc improved from 0.77833 to 0.79856, saving model to audio_weights/second-improvement-03-0.80.hdf5
Epoch 4/10
7636/7636 [==============================] - 13583s 2s/step - loss: 0.4110 - acc: 0.8886 - val_loss: 0.7120 - val_acc: 0.8088

Epoch 00004: val_acc improved from 0.79856 to 0.80877, saving model to audio_weights/second-improvem

# save training history

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'],loc='upper left')
plt.savefig('audioauc.png')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'],loc='upper left')
plt.savefig('audioloss.png')
plt.show()


In [9]:
#model.load_weights('weights-improvement-04-0.98.hdf5')
#model=load_model('audio_weights/first-improvement-10-0.82.hdf5')
model.evaluate_generator(audio.generate_arrays_from_file(testlist,batch_size=64),
                         steps=len(testlist)//64)

[0.793620569889362, 0.8155448717948718]